In [2]:
# ============================================================================
# 한국전력공사 변동계수 알고리즘 - 독립 실행용
# 1-2단계 결과 파일을 로드해서 실행
# ============================================================================

import pandas as pd
import numpy as np
import json
import os
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.preprocessing import StandardScaler, RobustScaler
import warnings
import joblib
from datetime import datetime
warnings.filterwarnings('ignore')

class KEPCOVolatilityCoefficient:
    """
    한국전력공사 전력 사용패턴 변동계수 스태킹 알고리즘
    1-2단계 결과 파일을 자동으로 로드해서 실행
    """
    
    def __init__(self, results_path='./analysis_results'):
        """
        초기화
        Args:
            results_path: 1-2단계 결과가 저장된 폴더 경로
        """
        self.results_path = results_path
        
        # 기본 설정 (1-2단계 결과로 자동 업데이트됨)
        self.config = {
            'temporal_weights': {
                'peak_hours': [9, 10, 11, 14, 15, 18, 19, 20],  
                'peak_weight': 1.5,                               
                'off_peak_weight': 0.8                            
            },
            'seasonal_adjustment': {
                'summer_months': [6, 7, 8],      
                'winter_months': [12, 1, 2],     
                'summer_factor': 1.2,            
                'winter_factor': 1.1             
            },
            'industry_baselines': {
                '222': 0.25,  # 일반용(갑)‖고압A
                '226': 0.30,  # 일반용(을) 고압A  
                '311': 0.35,  # 산업용(갑) 저압
                '322': 0.20,  # 산업용(갑)‖고압A
                '726': 0.28   # 산업용(을) 고압A
            },
            'usage_type_factors': {
                '02': 1.1,    # 상업용
                '09': 0.9     # 광공업용
            },
            'anomaly_thresholds': {
                'cv_extreme': 1.0,           
                'zero_ratio_max': 0.1,       
                'night_day_ratio_max': 0.8   
            }
        }
        
        # 모델 구성요소
        self.level0_models = {}
        self.level1_model = None
        self.scalers = {}
        self.is_fitted = False
        
        # 과적합 방지 설정
        self.cv_folds = 5
        self.random_state = 42
        
        print("🔧 한국전력공사 변동계수 알고리즘 초기화")
        print(f"결과 폴더: {self.results_path}")
        
        # 🔥 핵심: 1-2단계 결과 자동 로드
        self._load_previous_results()
    
    def _load_previous_results(self):
        """
        1-2단계 결과 파일들을 자동으로 로드해서 설정 업데이트
        """
        print("📂 1-2단계 결과 파일 로드 중...")
        
        try:
            # 1단계 결과 로드
            step1_file = os.path.join(self.results_path, 'analysis_results.json')
            if os.path.exists(step1_file):
                with open(step1_file, 'r', encoding='utf-8') as f:
                    step1_results = json.load(f)
                print("✅ 1단계 결과 로드 완료")
                self._update_config_from_step1(step1_results)
            else:
                print(f"⚠️ 1단계 결과 파일 없음: {step1_file}")
            
            # 2단계 결과 로드
            step2_file = os.path.join(self.results_path, 'volatility_summary.csv')
            if os.path.exists(step2_file):
                step2_results = pd.read_csv(step2_file)
                print("✅ 2단계 결과 로드 완료")
                self._update_config_from_step2(step2_results)
            else:
                print(f"⚠️ 2단계 결과 파일 없음: {step2_file}")
                
        except Exception as e:
            print(f"⚠️ 결과 파일 로드 중 오류: {e}")
            print("기본 설정으로 진행합니다.")
    
    def _update_config_from_step1(self, step1_results):
        """1단계 결과로 설정 업데이트"""
        print("🔄 1단계 결과로 설정 업데이트...")
        
        # 고객 분포 정보
        customer_summary = step1_results.get('customer_summary', {})
        if customer_summary:
            total_customers = customer_summary.get('total_customers', 3000)
            print(f"   고객 수: {total_customers:,}명")
            
            # 계약종별 분포로 기준값 조정
            contract_types = customer_summary.get('contract_types', {})
            if contract_types:
                print(f"   계약종별: {len(contract_types)}개 유형")
                
                # 각 계약종별 비율에 따라 기준값 미세 조정
                total_contracts = sum(contract_types.values())
                for contract, count in contract_types.items():
                    if str(contract) in self.config['industry_baselines']:
                        ratio = count / total_contracts
                        # 비율이 높은 계약종별은 기준값을 약간 낮춤 (더 엄격하게)
                        if ratio > 0.3:  # 30% 이상 차지하는 경우
                            self.config['industry_baselines'][str(contract)] *= 0.95
    
    def _update_config_from_step2(self, step2_results):
        """2단계 결과로 설정 업데이트"""
        print("🔄 2단계 결과로 설정 업데이트...")
        
        # volatility_summary.csv에서 핵심 지표 추출
        for _, row in step2_results.iterrows():
            metric = row['metric']
            value = row['value']
            
            if metric == 'overall_cv':
                # 전체 변동계수를 기준으로 업종별 기준값 조정
                baseline_adjustment = value / 0.25  # 기본 0.25 대비
                for contract_type in self.config['industry_baselines']:
                    self.config['industry_baselines'][contract_type] *= baseline_adjustment
                print(f"   전체 CV 기준 조정: {baseline_adjustment:.3f}배")
                
            elif metric == 'weekday_cv' and 'weekend_cv' in step2_results['metric'].values:
                # 주말/평일 변동성 차이
                weekend_row = step2_results[step2_results['metric'] == 'weekend_cv']
                if not weekend_row.empty:
                    weekend_cv = weekend_row['value'].iloc[0]
                    weekend_factor = weekend_cv / value if value > 0 else 1.0
                    self.config['weekend_factor'] = weekend_factor
                    print(f"   주말 팩터: {weekend_factor:.3f}")
        
        print("✅ 설정 업데이트 완료")
    
    def load_actual_data_for_features(self):
        """
        실제 LP 데이터를 로드해서 특성 생성
        (1-2단계에서 전처리된 데이터 활용)
        """
        print("📊 실제 데이터로 특성 생성...")
        
        try:
            # 🔥 여기서 실제 LP 데이터를 로드해야 함
            # 방법 1: 1-2단계에서 전처리된 데이터가 저장되어 있다면
            processed_data_file = os.path.join(self.results_path, 'processed_lp_data.csv')
            
            if os.path.exists(processed_data_file):
                print("📂 전처리된 LP 데이터 로드...")
                lp_data = pd.read_csv(processed_data_file)
                lp_data['LP 수신일자'] = pd.to_datetime(lp_data['LP 수신일자'])
            else:
                # 방법 2: 원본 LP 데이터를 다시 로드 (1-2단계와 동일한 방식)
                print("📂 원본 LP 데이터 로드...")
                lp_data = self._load_original_lp_data()
            
            # 고객 데이터도 로드
            customer_file = os.path.join(os.path.dirname(self.results_path), '제13회 산업부 공모전 대상고객.xlsx')
            if os.path.exists(customer_file):
                customer_data = pd.read_excel(customer_file)
            else:
                customer_data = None
                print("⚠️ 고객 데이터 파일 없음")
            
            # 특성 생성
            features_dict = self.create_features(lp_data, customer_data)
            
            print(f"✅ 특성 생성 완료: {len(features_dict)}명")
            return features_dict
            
        except Exception as e:
            print(f"❌ 데이터 로드 실패: {e}")
            print("샘플 데이터로 테스트 실행...")
            return self._create_sample_features()
    
    def _load_original_lp_data(self):
        """원본 LP 데이터 로드 (1-2단계와 동일)"""
        import glob
        
        base_path = os.path.dirname(self.results_path)
        lp_files = glob.glob(os.path.join(base_path, 'processed_LPData_*.csv'))
        
        if not lp_files:
            raise FileNotFoundError("LP 데이터 파일을 찾을 수 없습니다.")
        
        dataframes = []
        for file_path in sorted(lp_files):
            df = pd.read_csv(file_path)
            
            # 컬럼명 표준화
            if 'LP수신일자' in df.columns:
                df = df.rename(columns={'LP수신일자': 'LP 수신일자'})
            if '순방향유효전력' in df.columns:
                df = df.rename(columns={'순방향유효전력': '순방향 유효전력'})
            
            dataframes.append(df)
        
        lp_data = pd.concat(dataframes, ignore_index=True)
        lp_data['LP 수신일자'] = pd.to_datetime(lp_data['LP 수신일자'])
        
        return lp_data
    
    def _create_sample_features(self):
        """샘플 특성 데이터 생성 (테스트용)"""
        print("🧪 샘플 특성 데이터 생성...")
        
        sample_features = {}
        np.random.seed(42)
        
        for i in range(100):  # 100명 샘플
            customer_id = f'SAMPLE_{i:04d}'
            
            sample_features[customer_id] = {
                'basic': {
                    'cv_basic': np.random.normal(0.25, 0.1),
                    'range_volatility': np.random.normal(0.8, 0.3),
                    'iqr_volatility': np.random.normal(0.6, 0.2),
                    'skewness': np.random.normal(0.5, 0.3),
                    'kurtosis': np.random.normal(0.2, 0.5),
                    'mean_power': np.random.normal(50, 20),
                    'load_factor': np.random.normal(0.6, 0.2)
                },
                'temporal': {
                    'peak_cv': np.random.normal(0.3, 0.1),
                    'off_peak_cv': np.random.normal(0.2, 0.1),
                    'peak_mean': np.random.normal(60, 25),
                    'off_peak_mean': np.random.normal(40, 15),
                    'peak_off_peak_ratio': np.random.normal(1.5, 0.3),
                    'temporal_cv_diff': np.random.normal(0.1, 0.05),
                    'weekday_weekend_cv_ratio': np.random.normal(1.2, 0.3)
                },
                'seasonal': {
                    'summer_cv': np.random.normal(0.35, 0.15),
                    'winter_cv': np.random.normal(0.3, 0.12),
                    'summer_mean': np.random.normal(65, 30),
                    'winter_mean': np.random.normal(55, 25),
                    'seasonal_cv_diff': np.random.normal(0.05, 0.03),
                    'seasonal_stability': np.random.normal(0.1, 0.05)
                },
                'pattern': {
                    'daily_cv_stability': np.random.normal(0.05, 0.02),
                    'daily_cv_mean': np.random.normal(0.25, 0.1),
                    'autocorrelation': np.random.normal(0.7, 0.2),
                    'trend_volatility': np.random.normal(0.1, 0.05)
                },
                'anomaly': {
                    'zero_ratio': np.random.beta(1, 10),
                    'sudden_change_ratio': np.random.beta(1, 20),
                    'night_day_ratio': np.random.normal(0.4, 0.2),
                    'outlier_ratio': np.random.beta(1, 30)
                },
                'customer': {
                    'baseline_cv': 0.25,
                    'usage_factor': np.random.choice([0.9, 1.1]),
                    'contract_power_log': np.random.normal(6.0, 1.0)
                }
            }
        
        return sample_features
    
    # 나머지 메서드들은 이전 변동계수 알고리즘과 동일
    # (create_features, fit, predict, classify_volatility_grade 등)
    
    def create_features(self, lp_data, customer_data=None):
        """실제 LP 데이터로 특성 생성"""
        print("🔄 실제 데이터 특성 생성 중...")
        
        features_dict = {}
        customers = lp_data['대체고객번호'].unique()
        
        # 메모리 고려해서 최대 1000명으로 제한
        if len(customers) > 1000:
            customers = customers[:1000]
            print(f"   메모리 효율성을 위해 {len(customers)}명으로 제한")
        
        for i, customer in enumerate(customers):
            if (i + 1) % 100 == 0:
                print(f"   진행: {i+1}/{len(customers)} ({(i+1)/len(customers)*100:.1f}%)")
            
            customer_lp = lp_data[lp_data['대체고객번호'] == customer].copy()
            
            if len(customer_lp) < 96:  # 최소 1일 데이터 필요
                continue
            
            # 시간 관련 파생 변수
            customer_lp['시간'] = customer_lp['LP 수신일자'].dt.hour
            customer_lp['요일'] = customer_lp['LP 수신일자'].dt.weekday
            customer_lp['월'] = customer_lp['LP 수신일자'].dt.month
            customer_lp['주말여부'] = customer_lp['요일'].isin([5, 6])
            
            power_series = customer_lp['순방향 유효전력']
            
            # 1. 기본 변동성 특성
            basic_features = self._calculate_basic_volatility(power_series)
            
            # 2. 시간대별 변동성 특성
            temporal_features = self._calculate_temporal_volatility(customer_lp)
            
            # 3. 계절성 변동성 특성  
            seasonal_features = self._calculate_seasonal_volatility(customer_lp)
            
            # 4. 패턴 안정성 특성
            pattern_features = self._calculate_pattern_stability(customer_lp)
            
            # 5. 이상 패턴 특성
            anomaly_features = self._calculate_anomaly_features(customer_lp)
            
            # 6. 고객 특성
            customer_features = {}
            if customer_data is not None:
                customer_info = customer_data[customer_data['고객번호'] == customer]
                if not customer_info.empty:
                    customer_features = self._get_customer_features(customer_info.iloc[0])
                else:
                    customer_features = {'baseline_cv': 0.25, 'usage_factor': 1.0, 'contract_power_log': 6.0}
            else:
                customer_features = {'baseline_cv': 0.25, 'usage_factor': 1.0, 'contract_power_log': 6.0}
            
            features_dict[customer] = {
                'basic': basic_features,
                'temporal': temporal_features,
                'seasonal': seasonal_features,
                'pattern': pattern_features,
                'anomaly': anomaly_features,
                'customer': customer_features
            }
        
        print(f"✅ 특성 생성 완료: {len(features_dict)}명")
        return features_dict
    
    # 이하 _calculate_* 메서드들은 이전 코드와 동일하므로 생략
    # (실제 사용시에는 전체 메서드 포함)
    
    def run_complete_analysis(self):
        """전체 변동계수 분석 실행"""
        start_time = datetime.now()
        
        print("🚀 변동계수 알고리즘 분석 시작")
        print(f"시작 시간: {start_time}")
        
        try:
            # 1. 특성 데이터 준비
            features_dict = self.load_actual_data_for_features()
            
            if not features_dict:
                print("❌ 특성 데이터 준비 실패")
                return False
            
            # 2. 모델 학습
            print("🚀 스태킹 모델 학습...")
            training_results = self.fit(features_dict)
            
            # 3. 예측 수행
            print("🔮 변동계수 예측...")
            predictions = self.predict(features_dict)
            
            # 4. 결과 분석 및 리포트
            print("📋 최종 리포트 생성...")
            report_path = os.path.join(self.results_path, 'volatility_coefficient_report.json')
            final_report = self.generate_report(predictions, save_path=report_path)
            
            # 5. 모델 저장
            model_path = os.path.join(self.results_path, 'kepco_volatility_model.pkl')
            self.save_model(model_path)
            
            # 6. 등급별 분류 결과 저장
            self._save_classification_results(predictions)
            
            end_time = datetime.now()
            duration = end_time - start_time
            
            print("\\n" + "="*60)
            print("🏆 변동계수 분석 완료!")
            print("="*60)
            print(f"소요 시간: {duration}")
            print(f"분석 고객: {len(predictions)}명")
            print(f"평균 변동계수: {np.mean([p['volatility_coefficient'] for p in predictions.values()]):.4f}")
            
            # 등급별 분포 출력
            grades = {}
            for pred in predictions.values():
                grade_info = self.classify_volatility_grade(pred['volatility_coefficient'])
                grade = grade_info['grade']
                grades[grade] = grades.get(grade, 0) + 1
            
            print("\\n🎯 변동성 등급별 분포:")
            for grade, count in grades.items():
                pct = count / len(predictions) * 100
                print(f"   {grade}: {count}명 ({pct:.1f}%)")
            
            print(f"\\n📁 결과 파일:")
            print(f"   ✅ 모델: {model_path}")
            print(f"   ✅ 리포트: {report_path}")
            print(f"   ✅ 분류결과: {os.path.join(self.results_path, 'customer_volatility_grades.csv')}")
            
            return True
            
        except Exception as e:
            print(f"❌ 분석 실패: {e}")
            import traceback
            traceback.print_exc()
            return False
    
    def _save_classification_results(self, predictions):
        """고객별 변동성 등급 분류 결과 저장"""
        results = []
        
        for customer_id, pred in predictions.items():
            grade_info = self.classify_volatility_grade(pred['volatility_coefficient'])
            
            results.append({
                '고객번호': customer_id,
                '변동계수': pred['volatility_coefficient'],
                '변동성등급': grade_info['grade'],
                '위험수준': grade_info['risk_level'],
                '상대변동계수': grade_info['relative_cv'],
                '기준변동계수': grade_info['baseline_cv'],
                '신뢰도': pred['confidence_score'],
                '권장사항': grade_info['recommendation']
            })
        
        results_df = pd.DataFrame(results)
        output_path = os.path.join(self.results_path, 'customer_volatility_grades.csv')
        results_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        
        print(f"💾 분류 결과 저장: {output_path}")

# 실행 함수
def main():
    """메인 실행 함수"""
    print("=" * 60)
    print("한국전력공사 변동계수 알고리즘 - 독립 실행")
    print("=" * 60)
    
    # 결과 폴더 확인
    results_folders = ['./analysis_results', './results', './output']
    results_path = None
    
    for folder in results_folders:
        if os.path.exists(folder):
            results_path = folder
            break
    
    if results_path is None:
        print("⚠️ 결과 폴더를 찾을 수 없습니다.")
        print("다음 중 하나의 폴더에 1-2단계 결과를 저장하세요:")
        for folder in results_folders:
            print(f"   - {folder}")
        results_path = './analysis_results'
        os.makedirs(results_path, exist_ok=True)
        print(f"✅ 기본 폴더 생성: {results_path}")
    
    # 알고리즘 실행
    algorithm = KEPCOVolatilityCoefficient(results_path=results_path)
    success = algorithm.run_complete_analysis()
    
    if success:
        print("\\n🎉 변동계수 분석이 성공적으로 완료되었습니다!")
    else:
        print("\\n💥 변동계수 분석 중 오류가 발생했습니다.")

if __name__ == "__main__":
    main()

한국전력공사 변동계수 알고리즘 - 독립 실행
⚠️ 결과 폴더를 찾을 수 없습니다.
다음 중 하나의 폴더에 1-2단계 결과를 저장하세요:
   - ./analysis_results
   - ./results
   - ./output
✅ 기본 폴더 생성: ./analysis_results
🔧 한국전력공사 변동계수 알고리즘 초기화
결과 폴더: ./analysis_results
📂 1-2단계 결과 파일 로드 중...
⚠️ 1단계 결과 파일 없음: ./analysis_results\analysis_results.json
⚠️ 2단계 결과 파일 없음: ./analysis_results\volatility_summary.csv
🚀 변동계수 알고리즘 분석 시작
시작 시간: 2025-07-08 15:24:31.688578
📊 실제 데이터로 특성 생성...
📂 원본 LP 데이터 로드...
❌ 데이터 로드 실패: LP 데이터 파일을 찾을 수 없습니다.
샘플 데이터로 테스트 실행...
🧪 샘플 특성 데이터 생성...
🚀 스태킹 모델 학습...
❌ 분석 실패: 'KEPCOVolatilityCoefficient' object has no attribute 'fit'
\n💥 변동계수 분석 중 오류가 발생했습니다.


Traceback (most recent call last):
  File "C:\Users\khmin\AppData\Local\Temp\ipykernel_28056\3077651523.py", line 376, in run_complete_analysis
    training_results = self.fit(features_dict)
                       ^^^^^^^^
AttributeError: 'KEPCOVolatilityCoefficient' object has no attribute 'fit'
